In [3]:
import os
from pprint import pprint
from langchain_core.messages import AIMessage, HumanMessage, AnyMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain_core.tools import tool

In [4]:
load_dotenv()

True

In [5]:
messages = [AIMessage(content=f"So you said you were researching ocean mammals?", name="Model")]
messages.append(HumanMessage(content=f"Yes, that's right.", name="Jeel"))
messages.append(AIMessage(content=f"Great, what would you like to learn about.", name="Model"))
messages.append(HumanMessage(content=f"I want to learn about the best place to see Orcas in the US.", name="Jeel"))

for m in messages:
    m.pretty_print()

================================== Ai Message ==================================
Name: Model

So you said you were researching ocean mammals?
================================ Human Message =================================
Name: Jeel

Yes, that's right.
================================== Ai Message ==================================
Name: Model

Great, what would you like to learn about.
================================ Human Message =================================
Name: Jeel

I want to learn about the best place to see Orcas in the US.


In [6]:
api_key = os.getenv("GOOGLE_API_KEY")

In [7]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=api_key)
result = llm.invoke(messages)

In [8]:
type(result)

langchain_core.messages.ai.AIMessage

In [10]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b


llm_with_tools = llm.bind_tools([multiply])

result = llm_with_tools.invoke([HumanMessage(content=f"What is 2 multiplied by 3", name="Jeel")])

print("Tool calls:", result.tool_calls)

Tool calls: [{'name': 'multiply', 'args': {'b': 3, 'a': 2}, 'id': '2ef49a4d-3415-4abe-83d6-e86d1772b848', 'type': 'tool_call'}]


In [11]:
from typing import Annotated
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict
from langgraph.graph import MessagesState

In [12]:
class MessageState(MessagesState):
    pass

initial_messages = [AIMessage(content="Hello! How can I assist you?", name="Model"),
                    HumanMessage(content="I'm looking for information on langgraph.", name="Jeel")
                   ]

new_message = AIMessage(content="Sure, I can help with that. What specifically are you interested in?", name="Model")

add_messages(initial_messages, new_message)

[AIMessage(content='Hello! How can I assist you?', additional_kwargs={}, response_metadata={}, name='Model', id='e9a546f3-8550-4da0-8a36-b10d0be5705c'),
 HumanMessage(content="I'm looking for information on langgraph.", additional_kwargs={}, response_metadata={}, name='Jeel', id='5ab1ddea-9c94-4f93-8a19-8bec227b8e01'),
 AIMessage(content='Sure, I can help with that. What specifically are you interested in?', additional_kwargs={}, response_metadata={}, name='Model', id='4610b897-2de5-4dc6-a265-1862c3161443')]

In [13]:
from langgraph.graph import START,END,StateGraph

def tool_calling_llm(state: MessagesState):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

builder = StateGraph(MessageState)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_edge(START,"tool_calling_llm")
builder.add_edge("tool_calling_llm",END)
graph = builder.compile()

messages = graph.invoke({"messages": HumanMessage(content="Hello!")})
for m in messages['messages']:
    m.pretty_print()

messages = graph.invoke({"messages": HumanMessage(content="Multiply 2 and 3")})
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

Hello!
================================== Ai Message ==================================

Hello! How can I help you?
================================ Human Message =================================

Multiply 2 and 3
================================== Ai Message ==================================
Tool Calls:
  multiply (cfd468f0-9c66-46e8-a606-539b5779a0d2)
 Call ID: cfd468f0-9c66-46e8-a606-539b5779a0d2
  Args:
    b: 3
    a: 2
